This is where I keep all the functions for reading out the data

This function reads out the data

In [1]:
# function to read the data out of my stored data
def test_data_reader(test):
    resistor = 0
    distance_lamp_to_box = 0
    distance_box_to_detector = 0
    start_test = 0
    humidity = []
    dark_voltage = []
    light_voltage = []
    temperature = []
    voltage_gain = []

    for k, v in test.items():
        if k == "system_data":
            resistor = v[0]
            distance_lamp_to_box = v[1]
            distance_box_to_detector = v[2]
            start_test = v[3]
        else:
            actual_voltage = v[1] - v[0]
            humidity.append(k)
            dark_voltage.append(v[0])
            light_voltage.append(v[1])
            temperature.append(v[2])
            voltage_gain.append(actual_voltage)

    return resistor, distance_lamp_to_box, distance_box_to_detector, start_test, \
            humidity, dark_voltage, light_voltage, temperature, voltage_gain

# function to make nice colors for the plot
def color_magic(temp):
    temperature = temp
    min_temp = min(temperature)
    max_temp = max(temperature)
    difference = max_temp - min_temp

    some_dict = dict()
    color = []

    for T in temp:
        red_color = (T - min_temp) / difference
        # RGB = [red_color,red_color/1.5,0.545]
        RGB = [red_color, 0, red_color / 3]
        some_dict[T] = RGB
        color.append(RGB)

    return some_dict, color

Now follows the functions that I use for the simulations

We start with some basic functions and importing numpy

In [2]:
import numpy as np

'''basic functions'''
# function to measure the intensity left over after a certain path length
def measure_intensity(start_intensity, atten_coef, length):
    return start_intensity * np.exp(-atten_coef * length)

# returns the difference in dB between start signal and end signal
def intensity_in_dB(start_intensity, current_intensity):
    return 10 * np.log10(current_intensity / start_intensity)

def wavenumber_to_wavelength(WN):
    return 0.01 / (WN)

# returns water saturation at g/m3
def calculate_water_saturation(T):
    return np.exp(1.56065404) * np.exp(6.90219321e-02 * T) * np.exp(-2.38543241e-04 * T ** 2)

# returns air density at kg/m3
def calculate_air_density(T):
    return 1.44685243e-05 * T ** 2 - 4.72481940e-03 * T + 1.29255728

# returns decimal percentage of the max percentage water in the air at certain temperature T
# accurate between -20 and 60 C
def calculate_maximum_percentage_water(T):
    water_saturation = calculate_water_saturation(T)  # g/m3
    air_density = calculate_air_density(T)  # kg/m3

    max_percentage = (water_saturation / (air_density * 1000))  # decimal percentage

    return max_percentage

def make_2d_humidity_array(x=101, y=101):
    humidity = np.zeros((x,y))
    for i in range(x):
        for j in range(y):
            humidity[i,j] = j

    return humidity

# this function has to be square
def make_2d_path_length_array(x=101, y=101):
    path_length = np.zeros((x,y))
    for i in range(x):
        for j in range(y):
            path_length[i,j] = i/(y-1)

    return path_length

def make_2d_decimal_array(length_path):
    number_of_decimals = 3
    decimal_list = np.zeros((length_path,number_of_decimals))
    for i in range(length_path):
        for j in range(number_of_decimals):
            decimal_list[i,j] = j+3

    return decimal_list

# returns ampere for given intensity (mW/cm2) that the photodiode receives
def photodetector_1(intensity):
    return 37*intensity * 1e-6

Now the functions for the wikipedia data sets.

In [3]:
def WIKI_absorption_coefficient_air_composition_high_humidity(mu_H20):

    high_H2O = 0.03
    high_humidity = mu_H20 * high_H2O
    return high_humidity

def WIKI_find_humidity_from_intensity(start_intensity, received_intensity, path_length, mu_H2O):

    high_H2O = 0.03

    concentration_H2O = -((np.log(received_intensity / start_intensity)) / path_length) / mu_H2O

    percentage_humidity = (concentration_H2O / high_H2O) * 100

    return percentage_humidity


# now calculate absorption coefficient with the temperature
def WIKI_absorption_coefficient_air_composition_max_humidity_at_T(mu_H20, T):

    high_H2O = calculate_maximum_percentage_water(T)
    high_humidity = mu_H20 * high_H2O
    return high_humidity


def WIKI_find_humidity_from_intensity_at_T(start_intensity, received_intensity, path_length, mu_H2O, T):

    high_H2O = calculate_maximum_percentage_water(T)

    concentration_H2O = -((np.log(received_intensity / start_intensity)) / path_length) / mu_H2O

    percentage_humidity = (concentration_H2O / high_H2O) * 100

    return percentage_humidity

def WIKI_find_intensity_loss(start_intensity, path_length, mu_H2O, humidity, T):

    humidity = humidity/100

    max_percentage_water = calculate_maximum_percentage_water(T)
    attenuation_coefficient = mu_H2O*humidity*max_percentage_water
    intensity = measure_intensity(start_intensity, attenuation_coefficient, path_length)
    return intensity
